This notebook will help you define and run pipelines to process your data. This includes data augmentation, slicing, stretching and encoding among others. If you want to use this notebook, you are expected to have already collated your original `.xml` with the help of `1.1. Collate Files.ipynb`.

Pipelines are a data processing module which transforms input data types to output data types. The idea as well as bits & pieces are borrowed [the Magenta project](https://github.com/tensorflow/magenta/tree/master/magenta/pipelines).


**INSTRUCTIONS**
 
First, adjust the definition of the pipelines inside `pipeline_graph_def`. Then run `build_dataset`. This will create 4 files, two sets of train and evaluate. The first set is the inputs, and the second set is the targets.

**DEPENDENCIES**

In [1]:
from utils import data_processing
from utils.data_processing import PerformanceExtractor, PerformanceParser, TranspositionPipeline

from magenta.protobuf import music_pb2
from magenta.pipelines import pipelines_common, dag_pipeline, note_sequence_pipelines

import os
import re

**PARAMETERS**

In [10]:
pipeline_config = dict()

pipeline_config['data_source_dir'] = "./data/collated/A/"
pipeline_config['data_target_dir'] = "./data/processed/heron/"

# How many steps per quarter note
pipeline_config['steps_per_quarter'] = 4

pipeline_config['min_events'] = 1

pipeline_config['max_events'] = 10000

# Inclusive.
pipeline_config['MIN_MIDI_PITCH'] = 0

# Inclusive.
pipeline_config['MAX_MIDI_PITCH'] = 127

**DEFINITIONS**

In [13]:
def pipeline_graph_def(collection_name,
                       config):
    """Returns the Pipeline instance which creates the RNN dataset.

    Args:
        collection_name:
        config: dict() with configuration settings

    Returns:
        A pipeline.Pipeline instance.
    """
    
    # Transpose no more than a major third.
    transposition_range = range(-12, 12)
    
    train_mode = re.match(r'train(?=_)', collection_name)
    
    key = collection_name

    dag = {}
    
    quantizer = note_sequence_pipelines.Quantizer(
        steps_per_quarter=pipeline_config['steps_per_quarter'], 
        name='Quantizer_' + key)
        # `Quantizer` takes note data in seconds and snaps, or quantizes, 
        # everything to a discrete grid of timesteps. It maps `NoteSequence` 
        # protocol buffers to `NoteSequence` protos with quanitzed times. 
        #
        # input_type=music_pb2.NoteSequence
        # output_type=music_pb2.NoteSequence

    transposer = TranspositionPipeline(
        transposition_range if train_mode else [0],
        min_pitch = pipeline_config['MIN_MIDI_PITCH'],
        max_pitch = pipeline_config['MAX_MIDI_PITCH'],
        name='Transposer_' + key)
        # input_type=music_pb2.NoteSequence
        # output_type=music_pb2.NoteSequence

    perf_extractor = PerformanceExtractor(
        min_events=pipeline_config['min_events'],
        max_events=pipeline_config['max_events'],
        num_velocity_bins=0,
        name='PerformanceExtractor_' + key)
        # input_type = music_pb2.NoteSequence
        # output_type = magenta.music.MetricPerformance

    perf_parser = PerformanceParser(
        name='PerformanceParser_' + key)
        # input_type = magenta.music.MetricPerformance
        # output_type = str

    # Reverse
    # Meter
    
    # In > Quantize > Transpose > Extract Performance > Parse to text > Out
    dag[quantizer] = dag_pipeline.DagInput(music_pb2.NoteSequence)
    dag[transposer] = quantizer
    dag[perf_extractor] = transposer
    dag[perf_parser] = perf_extractor
    dag[dag_pipeline.DagOutput(key)] = perf_parser
        
    return dag_pipeline.DAGPipeline(dag)

# Build Dataset

In [14]:
data_processing.build_dataset(pipeline_config, pipeline_graph_def)


INFO: Building train_inputs dataset... Please wait...
Transposition range range(-12, 12)
Transposition pipeline ignores Key Signatures, Pitch Names and Chord Symbols.
INFO:tensorflow:

Completed.

INFO:tensorflow:Processed 2540 inputs total. Produced 60960 outputs.
INFO:tensorflow:DAGPipeline_PerformanceExtractor_train_inputs_performance_lengths_in_bars:
  [-inf,1): 264
  [1,10): 13008
  [10,20): 27192
  [20,30): 8760
  [30,40): 9480
  [40,50): 1584
  [50,100): 672
INFO:tensorflow:DAGPipeline_PerformanceExtractor_train_inputs_performances_discarded_more_than_1_program: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_train_inputs_performances_discarded_too_short: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_train_inputs_performances_truncated: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_train_inputs_performances_truncated_timewise: 0
INFO:tensorflow:DAGPipeline_Transposer_train_inputs_skipped_due_to_range_exceeded: 0
INFO:tensorflow:DAGPipeline_Transposer_train_inputs_t

# Build Vocabulary

In [15]:
data_processing.build_vocab(pipeline_config)

INFO: Vocabulary built.


AssertionError: 